In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# %matplotlib inline # plot in cell
from sklearn import metrics
import pandas as pd
from scipy import signal
from scipy import stats

import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# for svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC

In [2]:
import pandas as pd

data = pd.read_excel(r'F:\Program\OneDrive\TLNC\ISI\data\data.xlsx')
data.columns
data

Index(['Person', 'State', 'x', 'y', 'z'], dtype='object')

,Person,State,x,y,z
0,33.0,Jogging,-0.694638,12.680544,0.503953
1,33.0,Jogging,5.012288,11.264028,0.953424
2,33.0,Jogging,4.903325,10.882658,-0.081722
3,33.0,Jogging,-0.612916,18.496431,3.023717
4,33.0,Jogging,-1.184970,12.108489,7.205164
...,...,...,...,...,...
1048570,7.0,Standing,-8.080000,5.090000,-0.690000
1048571,7.0,Standing,-8.120000,5.520000,-0.800000
1048572,7.0,Standing,-7.890000,5.790000,-0.720000
1048573,7.0,Standing,-7.890000,5.980000,-0.270000


In [3]:
G = 9.81
data['x'] = data['x'].div(G)
data['y'] = data['y'].div(G)
data['z'] = data['z'].div(G)
data

,Person,State,x,y,z
0,33.0,Jogging,-0.070809,1.292614,0.051371
1,33.0,Jogging,0.510937,1.148219,0.097189
2,33.0,Jogging,0.499829,1.109343,-0.008330
3,33.0,Jogging,-0.062479,1.885467,0.308228
4,33.0,Jogging,-0.120792,1.234301,0.734471
...,...,...,...,...,...
1048570,7.0,Standing,-0.823649,0.518858,-0.070336
1048571,7.0,Standing,-0.827727,0.562691,-0.081549
1048572,7.0,Standing,-0.804281,0.590214,-0.073394
1048573,7.0,Standing,-0.804281,0.609582,-0.027523


In [4]:
data = data.drop(columns = 'Person')
df = data.groupby(['State'])
df.first()

,x,y,z
State,,,
Downstairs,-0.004165,0.508160,0.663662
Jogging,-0.070809,1.292614,0.051371
Sitting,0.265036,0.984709,0.105520
Standing,-0.191641,1.004077,-0.023603
Upstairs,1.031592,0.441516,-0.385979
Walking,0.086082,0.816388,0.284625


In [5]:
Downstairs = df.get_group(('Downstairs'))
Jogging = df.get_group(('Jogging'))
Sitting = df.get_group(('Sitting'))
Standing = df.get_group(('Standing'))
Upstairs = df.get_group(('Upstairs'))
Walking = df.get_group(('Walking'))

len(Downstairs)
len(Jogging)
len(Sitting)
len(Standing)
len(Upstairs)
len(Walking)

97813

325975

54876

44541

118589

406776

In [6]:
stand = Standing
len(stand)
sit = Sitting
len(sit)
jog = Jogging
len(jog)
walk = Walking
len(walk)
down = Downstairs
len(down)
up = Upstairs
len(up)

44541

54876

325975

406776

97813

118589

In [7]:
# Split dataset to 2 parts: Train (60%) - Test (40%)\n",
# 200 samples/frame

window_size = 200
stride = 100 #step

# range (start, stop, step)
X_stand_train = [stand[i:i+window_size] for i in range(0, int(len(stand)*0.6), stride)] 
X_stand_test = [stand[i:i+window_size] for i in range(int(len(stand)*0.6), len(stand), stride)
                                                    if i+window_size<=len(stand)]

X_sit_train = [sit[i:i+window_size] for i in range(0, int(len(sit)*0.6), stride)]
X_sit_test = [sit[i:i+window_size] for i in range(int(len(sit)*0.6), len(sit),stride)
                                           if i+window_size<=len(sit)]

X_jog_train = [jog[i:i+window_size] for i in range(0, int(len(jog)*0.6), stride)] 
X_jog_test = [jog[i:i+window_size] for i in range(int(len(jog)*0.6), len(jog), stride) 
                                            if i+window_size<=len(jog)]

X_walk_train = [walk[i:i+window_size] for i in range(0, int(len(walk)*0.6), stride)] 
X_walk_test = [walk[i:i+window_size] for i in range(int(len(walk)*0.6), len(walk), stride) 
                                            if i+window_size<=len(walk)]

X_down_train = [down[i:i+window_size] for i in range(0, int(len(down)*0.6), stride)] 
X_down_test = [down[i:i+window_size] for i in range(int(len(down)*0.6), len(down), stride) 
                                            if i+window_size<=len(down)]

X_up_train = [up[i:i+window_size] for i in range(0, int(len(up)*0.6), stride)] 
X_up_test = [up[i:i+window_size] for i in range(int(len(up)*0.6), len(up), stride) 
                                            if i+window_size<=len(up)]

print ('X_stand_train: ', len(X_stand_train))
print ('X_stand_test: ', len(X_stand_test))

print ('X_sit_train: ', len(X_sit_train))
print ('X_sit_test: ', len(X_sit_test))

print ('X_jog_train: ', len(X_jog_train))
print ('X_jog_test: ', len(X_jog_test))

print ('X_walk_train: ', len(X_walk_train))
print ('X_walk_test: ', len(X_walk_test))

print ('X_down_train: ', len(X_down_train))
print ('X_down_test: ', len(X_down_test))

print ('X_up_train: ', len(X_up_train))
print ('X_up_test: ', len(X_up_test))

X_stand_train:  268
X_stand_test:  177
X_sit_train:  330
X_sit_test:  218
X_jog_train:  1956
X_jog_test:  1302
X_walk_train:  2441
X_walk_test:  1626
X_down_train:  587
X_down_test:  390
X_up_train:  712
X_up_test:  473


In [8]:
train_data = []
train_label = []

test_data = []
test_label = []

for acts in X_stand_train:
    train_data.append(acts)
    train_label.extend([0]*120)
    
for acts in X_sit_train:
    train_data.append(acts)
    train_label.extend([1]*120)
    
for acts in X_jog_train:
    train_data.append(acts)
    train_label.extend([2]*120)

for acts in X_walk_train:
    train_data.append(acts)
    train_label.extend([3]*120)
    
for acts in X_down_train:
    train_data.append(acts)
    train_label.extend([4]*120)
    
for acts in X_up_train:
    train_data.append(acts)
    train_label.extend([5]*120)
    
    
print('train-data length: ', len(train_data) )
print('train-label length: ', len(train_label) )
#print(train_label)
      
# For TEST

for acts in X_stand_test:
    test_data.append(acts)
    test_label.extend([0]*120)

for acts in X_sit_test:
    test_data.append(acts)
    test_label.extend([1]*120)
        
for acts in X_jog_test:
    test_data.append(acts)
    test_label.extend([2]*120)

for acts in X_walk_test:
    test_data.append(acts)
    test_label.extend([3]*120)
    
for acts in X_down_test:
    test_data.append(acts)
    test_label.extend([4]*120)
    
for acts in X_up_test:
    test_data.append(acts)
    test_label.extend([5]*120)

print('test-data length: ', len(test_data))
print('test-label length: ', len(test_label))

train-data length:  6294
train-label length:  755280
test-data length:  4186
test-label length:  502320


In [9]:
import numpy as np
from PyMoments import kstat
import pandas as pd
import itertools

train_features = []
test_features = []
third_array = list(itertools.product([0,1,2], repeat=3))
fourth_array = list(itertools.product([0,1,2], repeat=4))

#TRAIN
for index_train in train_data: 
    convert_train = pd.DataFrame.to_numpy(index_train)
        
    first_order_kstats_train = [
        kstat(convert_train, (0,)),  
        kstat(convert_train, (1,)),
        kstat(convert_train, (2,))]
  
    train_features.extend(first_order_kstats_train)
    
    second_order_kstats_train = np.zeros((3, 3))
    for i in range(3):
        for j in range(3):
            second_order_kstats_train[i, j] = kstat(convert_train, (i, j))
            
    for i in range(3):
        for j in range(3):
            train_features.append(second_order_kstats_train[i][j])
            
    #third-order
    for index in third_array:
        train_features.append(kstat(convert_train, index)) 
     
    #fourth-order
    for idx in fourth_array:
        train_features.append(kstat(convert_train, idx)) 
        
train_features = np.array(train_features)


#TEST 

for index_test in test_data: 
    convert_test = pd.DataFrame.to_numpy(index_test)
        
    first_order_kstats_test = [
        kstat(convert_test, (0,)),  
        kstat(convert_test, (1,)),
        kstat(convert_test, (2,))]
    
    test_features.extend(first_order_kstats_test)
    
    second_order_kstats_test = np.zeros((3, 3))
    for i in range(3):
        for j in range(3):
            second_order_kstats_test[i, j] = kstat(convert_test, (i, j))
            
    for i in range(3):
        for j in range(3):
            test_features.append(second_order_kstats_test[i][j])
            
    #third-order
    for index in third_array:
        test_features.append(kstat(convert_test, index))
        
    #fourth-order    
    for idx in fourth_array:
        test_features.append(kstat(convert_test, idx))
            
test_features = np.array(test_features)

train_features
test_features


array([ 0.07680938,  1.00994393, -0.14833822, ..., -0.01488054,
        0.02247188,  0.0395665 ])

array([-3.29378186e-01,  8.63093782e-01, -1.18272171e-01, ...,
        7.31925392e-04, -7.79582976e-03,  1.05414299e-02])

In [10]:
len(train_features)
len(test_features)

755280

502320

In [11]:
# len(train_features) = len(train_label)

train_features.shape
train_features = train_features.reshape(-1, 1)
train_features.shape

test_features.shape
test_features = test_features.reshape(-1, 1)
test_features.shape


type(train_features)
len(train_features)
len(train_label)

len(test_features)
len(test_label)

(755280,)

(755280, 1)

(502320,)

(502320, 1)

numpy.ndarray

755280

755280

502320

502320

In [12]:
#check nan value

#np.where(np.isnan(train_features), dtype=np.float64) 
train_features = np.nan_to_num(train_features)
#np.where(np.isnan(test_features), dtype=np.float64)
test_features = np.nan_to_num(test_features)      

len(train_features)
len(test_features)

755280

502320

In [ ]:
#Classifier 

from sklearn.feature_selection import SelectFromModel
print("Gradient Boosting Decision Tree:")
from sklearn.ensemble import GradientBoostingClassifier
clf1 = GradientBoostingClassifier(learning_rate=0.05,max_depth=3,n_estimators=100).fit(train_features, train_label)
#format: pass score in {:.3f}
print('Accuracy of GBDT classifier on training set: {:.3f}'
     .format(clf1.score(train_features, train_label)))
print('Accuracy of GBDT classifier on test set: {:.3f}'
     .format(clf1.score(test_features, test_label)))

print("\n\nDecision Tree:") 
from sklearn.tree import DecisionTreeClassifier
from adspy_shared_utilities import plot_decision_tree 

clf2 = DecisionTreeClassifier(max_depth=4).fit(train_features, train_label)

print('Accuracy of Decision Tree classifier on training set: {:.3f}'.format(clf2.score(train_features, train_label)))
print('Accuracy of Decision Tree classifier on test set: {:.3f}'
.format(clf2.score(test_features, test_label)))


print("\n\nSVM:")
clf3 = SVC(C=100, gamma='scale').fit(train_features, train_label)
print("Accuracy on training set: {:.2f}".format(clf3.score(train_features, train_label)))
print("Accuracy on test set: {:.2f}".format(clf3.score(test_features, test_label)))


print('\n\n Random Forests: ')
from sklearn.ensemble import RandomForestClassifier
clf4 = RandomForestClassifier(n_estimators=200, random_state=0).fit(train_features, train_label)

print('Accuracy of RF classifier on training set: {:.3f}'
     .format(clf4.score(train_features, train_label)))
print('Accuracy of RF classifier on test set: {:.3f}'
     .format(clf4.score(test_features, test_label)))


print('\n\n KNeighbor: ')
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 5, weights = 'distance').fit(train_features, train_label)
print('Accuracy of K-NN classifier on training set: {:.2f}'
     .format(knn.score(train_features, train_label)))
print('Accuracy of K-NN classifier on test set: {:.2f}'
     .format(knn.score(test_features, test_label)))


Gradient Boosting Decision Tree:
Accuracy of GBDT classifier on training set: 0.569
Accuracy of GBDT classifier on test set: 0.538


Decision Tree:
Accuracy of Decision Tree classifier on training set: 0.522
Accuracy of Decision Tree classifier on test set: 0.493


SVM:
